# Reference Joins

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LineageLogic/LakeLogic/blob/main/examples/02_core_patterns/reference_joins/playbook.ipynb) 
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/LineageLogic/LakeLogic/blob/main/examples/02_core_patterns/reference_joins/playbook.ipynb)

## Business Scenario

Customer records are often missing context like region or opt-out status. Without reference tables, analytics and compliance checks are inconsistent. You need a repeatable way to enrich records using trusted dimensions.

## Value Proposition

- Centralize reference data and reuse it across pipelines
- Enrich raw records with standard dimensions
- Validate referential integrity in one place

---

## Goals

1. Load source records and reference tables
2. Join and enrich data using SQL transformations
3. Produce a clean, enriched output


## Step 1: Load Source Data

We will load customer data from the local CSV file.


In [ ]:
from pathlib import Path
import csv
from lakelogic import DataProcessor

BASE = Path.cwd()
contract_path = BASE / "contract.yaml"
if not contract_path.exists():
    candidate = BASE / "examples" / "02_core_patterns" / "reference_joins" / "contract.yaml"
    if candidate.exists():
        BASE = candidate.parent
        contract_path = candidate

data_path = BASE / "data" / "customers.csv"

def read_csv_rows(path, limit=5):
    rows = []
    with open(path, newline="", encoding="utf-8") as handle:
        reader = csv.DictReader(handle)
        for idx, row in enumerate(reader):
            rows.append(row)
            if idx + 1 >= limit:
                break
    return rows

def read_csv_dicts(path):
    with open(path, newline="", encoding="utf-8") as handle:
        return list(csv.DictReader(handle))

def preview_rows(rows, title=None):
    if title:
        print(title)
    if not rows:
        print("<no rows>")
        return
    for row in rows:
        print(row)

def preview_frame(df, title=None, limit=5):
    if title:
        print(title)
    if df is None:
        print("<empty>")
        return
    if hasattr(df, "show"):
        try:
            df.show(limit, truncate=False)
            return
        except Exception:
            pass
    if hasattr(df, "head"):
        try:
            head = df.head(limit)
            print(head)
            return
        except Exception:
            pass
    if hasattr(df, "limit"):
        try:
            sample = df.limit(limit)
            if hasattr(sample, "to_dicts"):
                for row in sample.to_dicts():
                    print(row)
                return
            print(sample)
            return
        except Exception:
            pass
    try:
        if isinstance(df, list):
            for row in df[:limit]:
                print(row)
            return
    except Exception:
        pass
    print(df)

def select_columns(df, columns):
    if hasattr(df, "select"):
        try:
            return df.select(columns)
        except Exception:
            pass
    try:
        if hasattr(df, "__getitem__"):
            return df[columns]
    except Exception:
        pass
    if isinstance(df, list):
        return [{col: row.get(col) for col in columns} for row in df]
    return df

if not data_path.exists():
    raise FileNotFoundError(f"Missing input file: {data_path.resolve()}")

preview_rows(read_csv_rows(data_path, limit=5), "RAW DATA (sample):")


## Step 2: Run the Contract

The contract.yaml file declares reference links and join logic.


In [ ]:
processor = DataProcessor(contract=contract_path)
source_rows = read_csv_dicts(data_path)
result = processor.run(source_rows, source_path=str(data_path))

good_df = result.good
bad_df = result.bad

report = processor.last_report or {}
print({"stage": report.get("stage"), "counts": report.get("counts")})


## Step 3: Inspect Results

Review enriched columns like region, opted_out, and full_name.


In [ ]:
selected = select_columns(good_df, ["email", "region", "opted_out", "full_name"])
preview_frame(selected, "ENRICHED OUTPUT (sample):")
